In [ ]:
# imports
import os
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from dataset import TrafficSignDataset
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
from matplotlib.patches import Rectangle
from matplotlib import image
from networks import TrafficSignsClassifier

from torchvision import transforms


In [ ]:
# test set transforms
img_size=32
dataset_mean = [86.72383685, 79.56345902, 81.93326525]
dataset_std= [51.48834219, 50.93286751, 53.30977311]

transform = transforms.Compose(
        [
            transforms.Resize((img_size, img_size)),
            transforms.Normalize(mean=dataset_mean, std=dataset_std)
        ]
)

In [ ]:
test = TrafficSignDataset("..\data\Test.csv", "..\data", transform=transform)

# to visualize predictions
unaugmented = TrafficSignDataset("..\data\Test.csv", "..\data", transform=None)

print(len(test))

In [ ]:
meta = pd.read_csv("..\data\Meta.csv",index_col=False, header=0)
meta = meta.set_index(meta["ClassId"])

### Visualize training samples

In [ ]:
cols, rows = 5, 6
idx = 0 # index of frame sequence

figure = plt.figure(figsize=(20, 24))

for i in range(idx * 30, idx * 30 + 30):
    img, target = test[i]

    figure.add_subplot(cols, rows, i - (idx * 30) + 1)

    plt.imshow(img.byte().permute(1,2,0))
    ax = plt.gca()

    # bbox = target["bbox"]
    # x = bbox[2]
    # y = bbox[3]
    # box_width = bbox[4] - x
    # box_height = bbox[5] - y

    # rect = Rectangle((x,y), box_width, box_height, linewidth=1, edgecolor='r',facecolor='none')

    # ax.add_patch(rect)
    plt.axis("off")

plt.show()

In [ ]:
batch_size = 256

# setup test loader
test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [ ]:
# setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

In [ ]:
# init model
model = TrafficSignsClassifier(None, 32, 43)

# Adam
model_name = "2_conv_2_k3_linear_relu_bn_maxp_b256_randp01_2022-01-22_20-12-26" 

# SGD 102 epochs
# model_name = "2_conv_2_k3_linear_relu_bn_maxp_b256_randp01_1e-3_val20_SGD2022-02-05_13-36-35"

# load trained params
model.load_state_dict(torch.load(f"..\models\{model_name}.pth"))

model.to(device)
model.eval()

In [ ]:
# define loss
criterion = nn.CrossEntropyLoss()

In [ ]:
size = len(test_dataloader.dataset)
num_batches = len(test_dataloader)
test_loss = 0
correct = 0.0

# test loop
with torch.no_grad():
    for i, data in enumerate(tqdm(test_dataloader)):
        # move data to device
        x, y = data
        x, y = x.to(device), y["label"].to(device)

        # prediction + loss
        logits = model(x)
        test_loss += criterion(logits, y).item()
        
        # accuracy
        preds = torch.argmax(logits, dim=1)
        correct += torch.sum(preds == y).item()

test_loss /= num_batches
correct /= size
print(f"Test Error: \n Accuracy: {(100*correct):>.2f}%, Avg loss: {test_loss:>.3f} \n")


### Visualize predictions

In [ ]:
cols, rows = 10, 6

figure = plt.figure(figsize=(20, 44))

ind = np.arange(60)
sample_subset = Subset(test, ind)
sample_dataloader = DataLoader(sample_subset, batch_size=60, shuffle=False)

for data in sample_dataloader:
    x, y = data
    x = x.to(device)

    logits = model(x)
    _, preds = torch.max(logits, 1)

    probs, top1s = torch.topk(torch.nn.functional.softmax(logits, dim=1, dtype=torch.float32), k=1, dim=1)

    probs = np.around(probs.cpu().detach().numpy()* 100, decimals=2)

print(preds)


for prob, pred, i in zip(probs, preds, sample_subset.indices):
    # show unaugmented images
    img, target = unaugmented[i]

    figure.add_subplot(cols, rows, i + 1)

    target_name = meta.loc[target['label']][5]
    pred_name = meta.loc[pred.cpu().numpy()][5]

    plt.title(f"Prediction: {pred_name} \n Probability: {prob[0]:.2f}%\n Target: {target_name}")

    plt.imshow(img.byte().permute(1,2,0))
    ax = plt.gca()

    # bbox = target["bbox"]
    # x = bbox[2]
    # y = bbox[3]
    # box_width = bbox[4] - x
    # box_height = bbox[5] - y

    # rect = Rectangle((x,y), box_width, box_height, linewidth=1, edgecolor='r',facecolor='none')

    # ax.add_patch(rect)
    plt.axis("off")

plt.show()

### Confusion matrix

In [ ]:
num_classes = 43

confusion_matrix = torch.zeros(num_classes, num_classes)
with torch.no_grad():
    for i, (x, y) in enumerate(test_dataloader):
        x = x.to(device)
        y = y["label"].to(device)
        outputs = model(x)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(y.view(-1), preds.view(-1)):
                confusion_matrix[t, p] += 1

print(confusion_matrix)

In [ ]:
import seaborn as sns

rows_sum = confusion_matrix.sum(axis=1)

norm_confusion_matrix = confusion_matrix / rows_sum[:, np.newaxis]

sns.set(rc={'figure.figsize':(24,20)})
ax = sns.heatmap(norm_confusion_matrix, annot=True, cmap='Blues', fmt='.1f')

plt.show()